In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import audio_tagging_utils as utils
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
import tensorflow.keras.backend as K
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Input
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.python.keras.engine import training
from tensorflow.python.framework.ops import Tensor
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, LeakyReLU, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.models import Sequential, Model



In [2]:
PREDICTION_FOLDER = "conv_pool_cnn_no_prepro"
if not os.path.exists(f'runs/{PREDICTION_FOLDER}'):
    os.mkdir(f'runs/{PREDICTION_FOLDER}')
if os.path.exists(f'runs/{PREDICTION_FOLDER}/logs'):
    shutil.rmtree(f'runs/{PREDICTION_FOLDER}/logs')

traindf=pd.read_csv('meta/train.csv')
testdf=pd.read_csv('meta/test.csv')
traindf["fname"]= traindf["fname"].apply(utils.append_ext)
testdf["fname"]= testdf["fname"].apply(utils.append_ext)

In [3]:
datagen=ImageDataGenerator(rescale=1./255.)
class_indices = {}

number_of_splits = 5

kfold_validation = KFold(n_splits = number_of_splits)

for i, (train_split_indexes, test_split_indexes) in enumerate(kfold_validation.split(traindf)):
    train_fold = traindf.iloc[train_split_indexes]
    val_fold = traindf.iloc[test_split_indexes]
    print(i)
    checkpoint = ModelCheckpoint(f'runs/{PREDICTION_FOLDER}/best_{i}.h5', monitor='val_loss', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
    tb = TensorBoard(log_dir=f'runs/{PREDICTION_FOLDER}/logs/fold_{i}', write_graph=True)

    callbacks_list = [checkpoint, early, tb]

    train_generator=datagen.flow_from_dataframe(
        dataframe=train_fold,
        directory="images/train_no_preprocessing/",
        x_col="fname",
        y_col="label",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=(64,64))

    valid_generator=datagen.flow_from_dataframe(
        dataframe=val_fold,
        directory="images/train_no_preprocessing/",
        x_col="fname",
        y_col="label",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=(64,64))

    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

0
Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
1
Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
2
Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
3
Found 7579 validated image filenames belonging to 41 classes.
Found 1894 validated image filenames belonging to 41 classes.
4
Found 7579 validated image filenames belonging to 41 classes.
Found 1894 validated image filenames belonging to 41 classes.


In [4]:
def conv_pool_cnn(model_input: Tensor, params: dict) -> training.Model:
    x = Conv2D(32, (3, 3), padding='same')(model_input)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][0])(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][1])(x)
    
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][2])(x)
    
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(params["dropouts"][3])(x)
    x = Dense(41, activation='softmax')(x)

    model = Model(model_input, x, name='conv_pool_cnn')
    model.compile(optimizers.Adam(params["lr"][0]),loss="categorical_crossentropy",metrics=['accuracy'])
    
    return model

In [13]:
model_input = Input(shape=(64, 64, 3))

#Initialise the parameters
params = {} 
params["dropouts"] = [0.5, 0.5, 0.5, 0.5] 
params["lr"] = [1.0]

conv_pool_cnn_model = conv_pool_cnn(model_input, params)

#Set Arbitrary placeholder variables
val_acc_max = 0
opt_dropout = 1
opt_lr = 10


#Optimise Learning Rate
while float(params["lr"][0]) > 0.0001:
    
    print('Optimising LR, LR = ' + str(params["lr"][0]) + '\n')
    
    #Fit the training data with one epoch
    optimise = conv_pool_cnn_model.fit(train_generator,
                steps_per_epoch=STEP_SIZE_TRAIN,
                validation_data=valid_generator,
                validation_steps=STEP_SIZE_VALID,
                epochs=1)
    
    #Collect the validation accuracy
    val_acc = optimise.history['val_accuracy']
    
    #Check if the validation accuracy is higher than the current best
    if float(val_acc[0]) > val_acc_max:
        
        #Update placeholders with the highest validation accuracy achieved
        #And the learning rate that achieved this
        val_acc_max = float(val_acc[0])
        opt_lr = params["lr"][0]
    
    print('\nvalidation accuracy for this run is: ' + str(val_acc[0]) + ' and highest accuracy achieved is: ' + str(val_acc_max) + '\n')
    #Update the learning rate to the next test sample
    params["lr"][0] *= 0.1

#Set the learning rate to the optimised value
params["lr"][0] = opt_lr


#Optimise Dropouts, for loop sets which layer we are optimising for
for i in range(0, len(params["dropouts"])):
    
    print('\nOptimising layer ' + str(i) + ' Droupout\n')
    
    #Set Arbitrary placeholder variables
    val_acc_max = 0
    opt_dropout = 1

    #Fit the training data with one epoch
    while params["dropouts"][i] > 0.05:

        optimise = conv_pool_cnn_model.fit(train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=1)
        #Collect the validation accuracyS
        
        val_acc = optimise.history['val_accuracy']
    
        #Check if the validation accuracy is higher than the current best
        if float(val_acc[0]) > val_acc_max:
            
            #Update placeholders with the highest validation accuracy achieved
            #And the dropout that achieved this
            val_acc_max = float(val_acc[0])
            opt_dropout = params["dropouts"][i]
        
        print('\nvalidation accuracy for this run is: ' + str(val_acc[0]) + ' and highest accuracy achieved is: ' + str(val_acc_max) + '\n')
        #Update the dropout to the next test sample
        params["dropouts"][i] -= 0.05
    
    #Set the dropout to the optimised value
    params["dropouts"][i] = opt_dropout
    

Optimising LR, LR = 1.0

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 236 steps, validate for 59 steps
236/236 [==============================] - 14s 60ms/step - loss: 34398172444965.9531 - accuracy: 0.0290 - val_loss: 3.9100 - val_accuracy: 0.0350

validation accuracy for this run is: 0.03495763 and highest accuracy achieved is: 0.034957628697156906

Optimising LR, LR = 0.1

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 236 steps, validate for 59 steps
236/236 [==============================] - 13s 55ms/step - loss: 3.8713 - accuracy: 0.0276 - val_loss: 3.8761 - val_accuracy: 0.0281

validation accuracy for this run is: 0.028072033 and highest accuracy achieved is: 0.034957628697156906

Optimising LR, LR = 0.010000000000000002

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 236 steps, validate for 59 steps
236/236 [==============================] - 13s 56ms/step - loss: 346417516.7130 - accuracy: 0.0278 - val_loss: 5.4415 - val_accuracy: 0.025

KeyboardInterrupt: 

In [ ]:
params